In [2]:
import os
import glob
import pandas as pd
from argparse import ArgumentParser
from sklearn.model_selection import GroupKFold

In [10]:
ALL_FILENAMES = glob.glob( '../dataset/npy_data/*.npy')
train_df = pd.read_csv("../dataset/train.csv")
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")

print(len(ALL_FILENAMES))
usable_names = set()
for n in ALL_FILENAMES:
    usable_names.add(os.path.split(n)[-1])
gkf = GroupKFold(n_splits=5)

120165


In [19]:
valid_seqid = set()
for fold_idx,(train, test) in enumerate(gkf.split(train_df.sequence_id, groups=train_df.participant_id)):
    for idx in test:
        valid_seqid.add(train_df.sequence_id[idx])
    train_info = []
    valid_info = []
    for seqid,label in zip(train_df.sequence_id,train_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names and seqid not in valid_seqid:
            train_info.append((fname,label))
    for seqid,label in zip(supp_df.sequence_id,supp_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names:
            train_info.append((fname,label))
    train_info = pd.DataFrame(train_info,columns=["files","labels"])

    for seqid,label in zip(train_df.sequence_id,train_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names and seqid in valid_seqid:
            valid_info.append((fname,label))
    valid_info = pd.DataFrame(valid_info,columns=["files","labels"])
    train_info.to_csv(f"../dataset/folds/fold{fold_idx}_train.csv",index=False)
    valid_info.to_csv(f"../dataset/folds/fold{fold_idx}_valid.csv",index=False)